# Тестовое задание

В данном задании необходимо написать код, который считает следующие признаки для каждого клиента `user_id`:

1. `events_ordinal_number` - порядковый номер события
2. `second_event_time` - время второго события
2.	`loan_ordinal_number` - порядковый номер займа
2.	`events_time_diff` - разница во времени между событиями
2.	`previous_loans_max_amount` - максимальная сумма предыдущего займа.


### Подготовка датафрейма

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
size = 10000

df = pd.DataFrame(
    {
        'time': [*pd.date_range('2001-01-01', '2020-09-01', freq='1h')][:size],
        'user_id': np.random.choice(1000, size),
        'type': np.random.choice(2, size, p=[0.8, 0.2]),
        'amount': [int(x//1) for x in  np.random.normal(15_000, 1_000, size)],
        'target': np.random.choice(2, size, p=[0.5, 0.5]),
    },
    index=[*range(size)],
)

In [3]:
df['is_loan'] = df['type']
df['type'] = df['type'].replace({0: 'loanRequest', 1: 'loan'})

In [4]:
df.to_csv('test_origin.csv', index=False)

### Расчет признаков

#### Признак `events_ordinal_number`

In [5]:
# весь код расчета признака должен быть в этом методе
def calculate_events_number(df):
    
    df['events_ordinal_number'] = df.groupby("user_id")["time"].rank().astype(int)
    
    return df

In [6]:
calculate_events_number(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number
0,2001-01-01 00:00:00,827,loanRequest,14696,1,0,1
1,2001-01-01 01:00:00,229,loan,16825,0,1,1
2,2001-01-01 02:00:00,501,loanRequest,14395,1,0,1
3,2001-01-01 03:00:00,878,loan,15878,1,1,1
4,2001-01-01 04:00:00,426,loanRequest,15276,1,0,1


In [7]:
df.to_csv('test_1st_check.csv', index = False)

#### Признак `second_event_time`

In [8]:
# весь код расчета признака должен быть в этом методе
def calculate_second_event_time(df):
    
    nth_user = df.groupby("user_id")[['time']].nth(1)
    nth_user.rename(columns = {'time': 'second_event_time'}, inplace = True)
    
    df = pd.merge(df, nth_user, on = ["user_id"])

    return df

In [9]:
df = calculate_second_event_time(df)

In [10]:
df.head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time
0,2001-01-01 00:00:00,827,loanRequest,14696,1,0,1,2001-01-03 03:00:00
1,2001-01-03 03:00:00,827,loan,16560,1,1,2,2001-01-03 03:00:00
2,2001-03-22 19:00:00,827,loanRequest,16493,1,0,3,2001-01-03 03:00:00
3,2001-04-16 02:00:00,827,loan,14677,0,1,4,2001-01-03 03:00:00
4,2001-04-26 22:00:00,827,loan,15302,0,1,5,2001-01-03 03:00:00


In [11]:
df.to_csv('test_2nd_check.csv', index = False)

#### Признак `loan_ordinal_number`

In [12]:
# весь код расчета признака должен быть в этом методе
def calculate_loan_number(df):

    df['loan_ordinal_number'] = df['is_loan'].apply(lambda x: 1 if x == 1 else 0)
    loan_labels = df['loan_ordinal_number'].copy()
    df['loan_ordinal_number'] = df.groupby('user_id')['loan_ordinal_number'].cumsum()+1
    df['loan_ordinal_number'] = df['loan_ordinal_number'] - loan_labels

    return df

In [13]:
calculate_loan_number(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time,loan_ordinal_number
0,2001-01-01 00:00:00,827,loanRequest,14696,1,0,1,2001-01-03 03:00:00,1
1,2001-01-03 03:00:00,827,loan,16560,1,1,2,2001-01-03 03:00:00,1
2,2001-03-22 19:00:00,827,loanRequest,16493,1,0,3,2001-01-03 03:00:00,2
3,2001-04-16 02:00:00,827,loan,14677,0,1,4,2001-01-03 03:00:00,2
4,2001-04-26 22:00:00,827,loan,15302,0,1,5,2001-01-03 03:00:00,3


In [14]:
df.to_csv('test_3rd_check.csv', index = False)

#### Признак `events_time_diff`

In [15]:
# весь код расчета признака должен быть в этом методе
def calculate_time_diff(df):
    df = df.sort_values(['user_id', 'events_ordinal_number'])
    
    df['events_time_diff'] = df.groupby('user_id')['time'].diff(1)

    return df

In [16]:
calculate_time_diff(df)

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff
9441,2001-04-22 06:00:00,0,loan,13542,0,1,1,2001-05-03 01:00:00,1,NaT
9442,2001-05-03 01:00:00,0,loanRequest,15417,0,0,2,2001-05-03 01:00:00,2,10 days 19:00:00
9443,2001-05-05 00:00:00,0,loanRequest,15298,1,0,3,2001-05-03 01:00:00,2,1 days 23:00:00
9444,2001-06-29 13:00:00,0,loanRequest,13086,1,0,4,2001-05-03 01:00:00,2,55 days 13:00:00
9445,2001-09-22 05:00:00,0,loanRequest,12594,1,0,5,2001-05-03 01:00:00,2,84 days 16:00:00
...,...,...,...,...,...,...,...,...,...,...
6825,2001-10-02 02:00:00,999,loanRequest,14138,0,0,5,2001-02-15 18:00:00,3,27 days 15:00:00
6826,2001-11-15 14:00:00,999,loan,15829,0,1,6,2001-02-15 18:00:00,3,44 days 12:00:00
6827,2001-11-28 04:00:00,999,loanRequest,15536,1,0,7,2001-02-15 18:00:00,4,12 days 14:00:00
6828,2001-12-14 08:00:00,999,loanRequest,13574,1,0,8,2001-02-15 18:00:00,4,16 days 04:00:00


In [17]:
df = calculate_time_diff(df)

In [18]:
df.to_csv('test_4th_check.csv', index = False)

#### Признак `previous_loans_max_amount`

In [19]:
# весь код расчета признака должен быть в этом методе
def calculate_previous_max_amount(df):

    def max_loan_amount(df2):
        series = []
        cur_result = np.nan
        series.append(cur_result)
        for i in df2.iloc[0:]:
            if np.isnan(i):
                series.append(cur_result)
                continue
            if np.isnan(cur_result):
                cur_result = i
                series.append(i)
                continue
            cur_result = max(cur_result, i)
            series.append(cur_result)

        return pd.Series(series[:-1], df2.index)

    df['previous_loans_max_amount'] = df['amount']
    df.loc[df['is_loan'] == 1, 'previous_loans_max_amount'] = np.nan
    df['previous_loans_max_amount'] = df.groupby('user_id')['previous_loans_max_amount'].apply(max_loan_amount)

    return df

In [20]:
calculate_previous_max_amount(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff,previous_loans_max_amount
9441,2001-04-22 06:00:00,0,loan,13542,0,1,1,2001-05-03 01:00:00,1,NaT,NaN
9442,2001-05-03 01:00:00,0,loanRequest,15417,0,0,2,2001-05-03 01:00:00,2,10 days 19:00:00,NaN
9443,2001-05-05 00:00:00,0,loanRequest,15298,1,0,3,2001-05-03 01:00:00,2,1 days 23:00:00,15417.0
9444,2001-06-29 13:00:00,0,loanRequest,13086,1,0,4,2001-05-03 01:00:00,2,55 days 13:00:00,15417.0
9445,2001-09-22 05:00:00,0,loanRequest,12594,1,0,5,2001-05-03 01:00:00,2,84 days 16:00:00,15417.0


In [21]:
df.head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff,previous_loans_max_amount
9441,2001-04-22 06:00:00,0,loan,13542,0,1,1,2001-05-03 01:00:00,1,NaT,NaN
9442,2001-05-03 01:00:00,0,loanRequest,15417,0,0,2,2001-05-03 01:00:00,2,10 days 19:00:00,NaN
9443,2001-05-05 00:00:00,0,loanRequest,15298,1,0,3,2001-05-03 01:00:00,2,1 days 23:00:00,15417.0
9444,2001-06-29 13:00:00,0,loanRequest,13086,1,0,4,2001-05-03 01:00:00,2,55 days 13:00:00,15417.0
9445,2001-09-22 05:00:00,0,loanRequest,12594,1,0,5,2001-05-03 01:00:00,2,84 days 16:00:00,15417.0


### Сохранение итогового датасета с признаками

Этот код трогать не нужно.

In [22]:
def calculate_feature(df):
    calculate_events_number(df)
    calculate_second_event_time(df)
    calculate_loan_number(df)
    calculate_time_diff(df)
    calculate_previous_max_amount(df)

In [23]:
%%timeit -o 
calculate_feature(df)

841 ms ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 841 ms ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [24]:
columns = ['time']
rez = _
df_time = pd.DataFrame([rez], None, columns)
df_time.to_csv('time.csv', index=False)

In [25]:
df.to_csv('test.csv', index=False)

### Проверка статистических гипотез о значимости признаков

In [26]:
df_check = pd.read_csv('test_Feb22.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

FileNotFoundError: [Errno 2] No such file or directory: 'test_Feb22.csv'

In [ ]:
df_check = df_check[['target', 'feature1', 'feature2']]

In [ ]:
df_check.head()

Нам предстоит работать с двумя бинарными переменными в качестве предикторов.
Несмотря на то, что наши данные довольно простые, проведём их первичный анализ и предобработку.

In [ ]:
def count_missing(df):
    mis_val = df_check.isnull().sum()
    mis_percent = 100 * mis_val / (len(df_check))
    result_table = pd.DataFrame(
        {
            "Missing values": mis_val,
            "% of all values": mis_percent
        }).sort_values("% of all values", ascending = False).round(2)
    return result_table

In [ ]:
mis_val_table_train = count_missing(df_check)
mis_val_table_train[mis_val_table_train["% of all values"] > 0]

In [ ]:
df_check['target'].value_counts()

Почти 80% целевых значений NaN. Мы могли бы предположить, что под NaN подразумеваются нули, однако при помощи value_counts мы выявили, что нулевое значение ннаписано цифрой.

Так как это переменная, которую мы прогнозируем, мы не можем заменить эти миссинги модой. Оставить всё как есть также не имеет смысла. Удалим наблюдения с пропущенным значением target, в датасете останется достаточно наблюдений для анализа.

In [ ]:
df_check.dropna(subset = ["target"], axis = 0, inplace = True)

In [ ]:
df_check.reset_index(drop = True, inplace = True)

In [ ]:
df_check.describe()

In [ ]:
df_check['feature1'].value_counts()

In [ ]:
df_check['feature2'].value_counts()

Фичи содержат в значениях только 0 и 1, мы это проверили, можно продолжать анализ

#### Кэф корреляции

Примем alpha = 0.05. 

In [ ]:
correlation_matrix = df_check.corr()
correlation_matrix

Видим, что коэффициент корреляции для feature1 равен 38%, это не так много, однако слабая взаимосвязь присутствует

In [ ]:
from scipy import stats
pearson_coef, p_value = stats.pearsonr(df_check['feature1'], df_check['target'])
pearson_coef, p_value

In [ ]:
from scipy import stats
pearson_coef, p_value = stats.pearsonr(df_check['feature2'], df_check['target'])
pearson_coef, p_value

p_value(feature2) = 0.0692, при принятом уровне значимости отвергаем гипотезу о значимости переменной feature2. Проверим переменные при помощи других методов.

#### Information Gain

Параметр, учитываемый при моделировании дерева решений. Дерево сплитит по той переменной, у которой выше IG. Соответственно, эта переменная эффективнее снижает степень неопределенности, которую мы стремимся свести к нулю.

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
X = df_check[['feature1', 'feature2']]
y = df_check.target

In [ ]:
importances = mutual_info_classif(X, y)

feature_importances = pd.Series(importances, df_check.columns[0:len(df_check.columns) - 1])
feature_importances.plot(kind = 'barh', color = 'teal')
plt.show()

#### Дерево решений

Построим дерево решений, а затем выявим более сильную переменную

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X,y)

In [ ]:
pred_train = clf.predict_proba(X)[:, 1]
print('ROC-AUC ', roc_auc_score(y.values, pred_train))

In [ ]:
feature_names = list(X.columns)
feature_importance = pd.DataFrame({'feature': feature_names,
                                   'importance': clf.feature_importances_})

In [ ]:
feature_importance.sort_values(by = 'importance', ascending=False)

In [ ]:
tree.plot_tree(clf, feature_names = list(X),
               class_names = ['Negative', 'Positive'],
               filled = True);

Оценим значение ROC-AUC для нашей модели

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
clf_test = RandomForestClassifier()
clf_test.fit(x_train, y_train)

pred_train = clf_test.predict_proba(x_train)[:, 1]
pred_val = clf_test.predict_proba(x_val)[:, 1]
print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

Для train ROC-AUC = 0.7021. Значит, она верно предскажет значение для 70% случаев в задаче классификации. Сильно хорошей такую модель не назовешь, однако значение больше 50%.

Построим модели на каждой переменной по отдельности. В реальности такое вряд ли встречается, однако это даст нам возможность сравнить силу переменных 

In [ ]:
X1 = X[['feature1']]
X2 = X[['feature2']]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X1, y, test_size=0.20, random_state=42)

clf_test = RandomForestClassifier()
clf_test.fit(x_train, y_train)

pred_train = clf_test.predict_proba(x_train)[:, 1]
pred_val = clf_test.predict_proba(x_val)[:, 1]
print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X2, y, test_size=0.20, random_state=42)

clf_test = RandomForestClassifier()
clf_test.fit(x_train, y_train)

pred_train = clf_test.predict_proba(x_train)[:, 1]
pred_val = clf_test.predict_proba(x_val)[:, 1]
print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

Если мы попробуем построить модель только на feature1: предсказтельная способность упадет, но не так значительно, - только на feature2: площадь под кривой будет равна ~50%, что почти свидетельствует о  рандомном выборе при предсказании.

Таким образом, выявлено, что вторая фича не даёт информации для целевой переменной: коэффициент Пирсона близок к 0,  при уровне значимости 5% переменная не значима, в перфомансе модели выступает как более слабая переменная. Feature1 же имеет слабую положительную корреляцию с целевой переменной, при построении дерева решений даёт больший IG (более 98%!), довольно самостоятельна в предсказании целевой переменной.